In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd as ag
from mxnet.gluon import nn

import numpy as np
import pandas as pd

/usr/local/lib/python3.5/dist-packages/_pytest/fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/usr/local/lib/python3.5/dist-packages/_pytest/fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


In [2]:
project_path = '/home/carsmart/users/xiaoming/Distracted_Driver_Detection/'
features_path = project_path + 'features/'
val_path_list = project_path + 'train_lst/train_val.lst'
path_list = project_path + 'train_lst/train_train.lst'
ctx = mx.gpu(0)
model_save_path = project_path + 'model/'

In [3]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis = 1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss / steps, acc / steps

## load features

In [4]:
def load_features(model_name, features_path, batch_size = 64):
    train_features = nd.load(features_path + 'features_train_%s.nd' % model_name)[0]
    val_features = nd.load(features_path + 'features_val_%s.nd' % model_name)[0]
    
    train_labels = nd.array(pd.read_table(path_list, header = None)[1].as_matrix())
    val_labels = nd.array(pd.read_table(val_path_list, header = None)[1].as_matrix())
    
    dataset_train = gluon.data.ArrayDataset(train_features, train_labels)
    dataset_val = gluon.data.ArrayDataset(val_features, val_labels)
    
    data_iter_train = gluon.data.DataLoader(dataset_train, batch_size)
    data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)
    
    return data_iter_train, data_iter_val

## model define

In [5]:
def get_net():
    net = nn.HybridSequential()
    with net.name_scope():
        net.add(
            nn.Dense(256, activation = 'relu'), 
            nn.Dropout(0.5), 
            nn.Dense(10)
        )
    net.hybridize()
    net.initialize(ctx = ctx)
    return net

## train define

In [6]:
def train(model_name):
    epochs = 50
    batch_size = 64
    
    data_iter_train, data_iter_val = load_features(model_name, features_path, batch_size)
    net = get_net()
    
    trainer = gluon.Trainer(net.collect_params(), 'adam', 
                            {'learning_rate': 1e-4, 'wd': 1e-5})
    
    for epoch in range(epochs):
        train_loss = 0.
        train_acc = 0.
        steps = len(data_iter_train)
        for data, label in data_iter_train:
            data, label = data.as_in_context(ctx), label.as_in_context(ctx)
            
            with ag.record():
                output = net(data)
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            
            train_loss += nd.mean(loss).asscalar()
            train_acc += accuracy(output, label)
            
        val_loss, val_acc = evaluate(net, data_iter_val)
    
    print('Epochs %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%% Model: %s' % 
          (epochs, train_loss / steps, train_acc / steps * 100, val_loss, val_acc * 100, model_name))
    
    return val_loss

## train

In [7]:
from mxnet.gluon.model_zoo.model_store import _model_sha1

losses = []
for model_name in sorted(_model_sha1.keys()):
    val_loss = train(model_name)
    losses.append((model_name, val_loss))

Epochs 50. loss: 0.0064, acc: 99.90%, val_loss 0.0340, val_acc 99.14% Model: alexnet
Epochs 50. loss: 0.0686, acc: 98.21%, val_loss 0.0659, val_acc 98.15% Model: densenet121
Epochs 50. loss: 0.0275, acc: 99.43%, val_loss 0.0364, val_acc 99.05% Model: densenet161
Epochs 50. loss: 0.0251, acc: 99.54%, val_loss 0.0377, val_acc 99.10% Model: densenet169
Epochs 50. loss: 0.0234, acc: 99.54%, val_loss 0.0354, val_acc 98.88% Model: densenet201
Epochs 50. loss: 0.0356, acc: 99.20%, val_loss 0.0593, val_acc 97.93% Model: inceptionv3
Epochs 50. loss: 0.0766, acc: 97.99%, val_loss 0.0857, val_acc 97.58% Model: resnet101_v1
Epochs 50. loss: 0.0820, acc: 97.91%, val_loss 0.0759, val_acc 98.09% Model: resnet152_v1
Epochs 50. loss: 0.0989, acc: 97.36%, val_loss 0.0896, val_acc 97.91% Model: resnet18_v1
Epochs 50. loss: 0.1086, acc: 97.20%, val_loss 0.0993, val_acc 97.25% Model: resnet18_v2
Epochs 50. loss: 0.1020, acc: 97.29%, val_loss 0.0951, val_acc 97.29% Model: resnet34_v1
Epochs 50. loss: 0.1109

## sort val_loss

In [8]:
val_loss = pd.DataFrame(losses, columns = ['model', 'val_loss'])
val_loss = val_loss.sort_values('val_loss')
val_loss.head(10)

,model,val_loss
16,vgg11,0.029661
0,alexnet,0.033955
14,squeezenet1.0,0.034491
4,densenet201,0.035379
2,densenet161,0.036374
15,squeezenet1.1,0.036922
18,vgg13,0.037571
3,densenet169,0.037724
22,vgg19,0.038942
19,vgg13_bn,0.039767


## 融合较好的几个模型

In [9]:
def load_models_features(model_names, batch_size = 64):
    train_features = [nd.load(features_path + 'features_train_%s.nd' % model_name)[0] for model_name in model_names]
    val_features = [nd.load(features_path + 'features_val_%s.nd' % model_name)[0] for model_name in model_names]
    train_features = nd.concat(*train_features, dim = 1)
    val_features = nd.concat(*val_features, dim = 1)
    
    train_labels = nd.array(pd.read_table(path_list, header = None)[1].as_matrix())
    val_labels = nd.array(pd.read_table(val_path_list, header = None)[1].as_matrix())
    
    dataset_train = gluon.data.ArrayDataset(train_features, train_labels)
    dataset_val = gluon.data.ArrayDataset(val_features, val_labels)
    
    data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle = True)
    data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)
    
    return data_iter_train, data_iter_val

## Fusion model train

In [10]:
net = get_net()
epochs = 100
batch_size = 64
lr_sch = mx.lr_scheduler.FactorScheduler(step = 400, factor = 0.9)
trainer = gluon.Trainer(net.collect_params(), 'adam', 
                        {'learning_rate': 1e-4, 'wd': 1e-5, 'lr_scheduler': lr_sch})

model_names = ['vgg11', 'densenet161', 'squeezenet1.0']
data_iter_train, data_iter_val = load_models_features(model_names, batch_size = batch_size)

for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        
        with ag.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    val_loss, val_acc = evaluate(net, data_iter_val)
    
    print('Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%' % 
          (epoch + 1, train_loss / steps, train_acc / steps * 100, val_loss, val_acc * 100))

Epoch 1. loss: 1.7639, acc: 42.17%, val_loss 0.8277, val_acc 86.22%
Epoch 2. loss: 0.7690, acc: 77.50%, val_loss 0.3753, val_acc 93.29%
Epoch 3. loss: 0.4504, acc: 88.31%, val_loss 0.2231, val_acc 95.88%
Epoch 4. loss: 0.3177, acc: 91.71%, val_loss 0.1689, val_acc 96.94%
Epoch 5. loss: 0.2392, acc: 94.19%, val_loss 0.1224, val_acc 97.78%
Epoch 6. loss: 0.1945, acc: 95.33%, val_loss 0.1021, val_acc 98.11%
Epoch 7. loss: 0.1634, acc: 96.12%, val_loss 0.0857, val_acc 98.37%
Epoch 8. loss: 0.1404, acc: 96.66%, val_loss 0.0757, val_acc 98.37%
Epoch 9. loss: 0.1233, acc: 96.92%, val_loss 0.0661, val_acc 98.55%
Epoch 10. loss: 0.1109, acc: 97.41%, val_loss 0.0595, val_acc 98.77%
Epoch 11. loss: 0.0976, acc: 97.83%, val_loss 0.0556, val_acc 98.86%
Epoch 12. loss: 0.0893, acc: 98.09%, val_loss 0.0503, val_acc 98.92%
Epoch 13. loss: 0.0827, acc: 98.08%, val_loss 0.0468, val_acc 98.99%
Epoch 14. loss: 0.0751, acc: 98.37%, val_loss 0.0449, val_acc 99.03%
Epoch 15. loss: 0.0691, acc: 98.52%, val_lo

In [11]:
net.save_params(model_save_path + '9947.params')